In [1]:
# first command tare

import socket
import threading
import datetime
import struct

# Specify server host and port
HOST = '192.168.0.129'
PORT = 8234

# Commands
GET_WEIGHT = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x06, 0x00, 0xac, 0x03]
TARE = [0x02, 0x01, 0x04, 0x00, 0x57, 0x50, 0x06, 0x00, 0xB2, 0x03]

getweight_counter = 0-1

def hexArrayToByteStream(hex_array):
    return bytes(hex_array)

def decode_weight(weight_bytes):
    # Interpret bytes as a signed 32-bit integer
    weight = struct.unpack('<i', bytes(weight_bytes))[0]
    weight_in_grams = weight * 0.1
    return weight_in_grams

def send_command_and_handle_response(client_socket, command):
    global getweight_counter
    print(f"getweight_counter: {getweight_counter}")
    
    response = hexArrayToByteStream(command)
    client_socket.sendall(response)
    
    msg = client_socket.recv(1024)
    time1 = datetime.datetime.now()
    
    if msg:
        hex_a = list(msg)
        weight0 = decode_weight(hex_a[8:-2][1:5])
        weight1 = decode_weight(hex_a[8:-2][-4:])
        print(f'weight 0: {weight0}| weight 1: {weight1}')
        print("continuous...")
    
    if command == GET_WEIGHT:
        getweight_counter += 1

def handle_client(client_socket):
    global getweight_counter
    
    # Perform initial TARE operation before starting the loop
    # print("Performing initial tare operation...")
    # send_command_and_handle_response(client_socket, TARE)
    
    while True:
        try:
            if getweight_counter % 1200 == 0 :
                print("Performing tare operation...")
                send_command_and_handle_response(client_socket, TARE)
                getweight_counter = 0
            
            send_command_and_handle_response(client_socket, GET_WEIGHT)
            
        except Exception as e:
            print(f"Error: {e}")
            if client_socket in client_sockets:
                client_sockets.remove(client_socket)
            client_socket.close()
            break

print("server ip:", HOST)
print("server port:", PORT)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(2)
print('Waiting for client connection...')

client_sockets = []

while True:
    client_socket, addr = server_socket.accept()
    print('Connecting to client {0}'.format(addr))
    client_sockets.append(client_socket)
    
    # Perform initial GET_WEIGHT command after connection
    send_command_and_handle_response(client_socket, GET_WEIGHT)
    print("Initial")
    
    thread = threading.Thread(target=handle_client, args=(client_socket,))
    thread.start()


server ip: 192.168.0.129
server port: 8234
Waiting for client connection...
Connecting to client ('192.168.0.8', 21007)
getweight_counter: -1
weight 0: 0.0| weight 1: 0.0
continuous...
Initial
Performing tare operation...
getweight_counter: 0
weight 0: 5941.900000000001| weight 1: 0.0
continuous...
getweight_counter: 0
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 1
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 2
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 3
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 4
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 5
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 6
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 7
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 8
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 9
weight 0: 0.0| weight 1: 0.0
continuous...
getweight_counter: 10
weight 0: 0.0| weight 1: 0.0
continuou

In [ ]:
products = {
    "CAN ": 275,  
    "PROCESSOR": 373,  
    
}


In [1]:
import socket
import threading
import datetime
import struct

# Specify server host and port
HOST = '192.168.0.129'
PORT = 8234

# Commands
GET_WEIGHT = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x06, 0x00, 0xac, 0x03]
TARE = [0x02, 0x01, 0x04, 0x00, 0x57, 0x50, 0x06, 0x00, 0xB2, 0x03]

getweight_counter = 0
previous_weights = {}

# Define the products and their weights
products = {
    "CAN": 275,
    "PROCESSOR": 373,
}

# Define the weight deviation threshold (5% in this case)
weight_deviation_threshold = 0.05

def hexArrayToByteStream(hex_array):
    return bytes(hex_array)

def decode_weight(weight_bytes):
    # Interpret bytes as a signed 32-bit integer
    weight = struct.unpack('<i', bytes(weight_bytes))[0]
    weight_in_grams = weight * 0.1
    return weight_in_grams

def send_command_and_handle_response(client_socket, command):
    global getweight_counter, previous_weights

    response = hexArrayToByteStream(command)
    client_socket.sendall(response)

    msg = client_socket.recv(1024)
    time1 = datetime.datetime.now()

    if msg:
        hex_a = list(msg)
        weight0 = decode_weight(hex_a[8:-2][1:5])
        weight1 = decode_weight(hex_a[8:-2][-4:])
        # print(f'weight 0: {weight0}| weight 1: {weight1}')

        # Check if an object has been picked or dropped for each product
        for product, good_weight in products.items():
            if product not in previous_weights:
                previous_weights[product] = 0

            if abs(weight0 - previous_weights[product]) > weight_deviation_threshold * good_weight:
                if weight0 > previous_weights[product]:
                    print(f"{product} picked")
                else:
                    print(f"{product} dropped")

            previous_weights[product] = weight0

        print("continuous...")

    if command == GET_WEIGHT:
        getweight_counter += 1

def handle_client(client_socket):
    global getweight_counter
    
    # Perform initial TARE operation before starting the loop
    # print("Performing initial tare operation...")
    # send_command_and_handle_response(client_socket, TARE)
    
    while True:
        try:
            if getweight_counter % 1200 == 0 :
                print("Performing tare operation...")
                send_command_and_handle_response(client_socket, TARE)
                getweight_counter = 0
            
            send_command_and_handle_response(client_socket, GET_WEIGHT)
            
        except Exception as e:
            print(f"Error: {e}")
            if client_socket in client_sockets:
                client_sockets.remove(client_socket)
            client_socket.close()
            break

print("server ip:", HOST)
print("server port:", PORT)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(2)
print('Waiting for client connection...')

client_sockets = []

while True:
    client_socket, addr = server_socket.accept()
    print('Connecting to client {0}'.format(addr))
    client_sockets.append(client_socket)
    
    # Perform initial GET_WEIGHT command after connection
    send_command_and_handle_response(client_socket, GET_WEIGHT)
    print("Initial")
    
    thread = threading.Thread(target=handle_client, args=(client_socket,))
    thread.start()



server ip: 192.168.0.129
server port: 8234
Waiting for client connection...
Connecting to client ('192.168.0.8', 21546)
continuous...
Initial
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
continuous...
cont

In [ ]:
import socket
import threading
import datetime
import time



In [ ]:

# pecify server host and port
# HOST = '192.168.0.201'
HOST = '192.168.0.129'
PORT = 8235


In [ ]:

def hexArrayToByteStream(hex_array):  # default
    return bytes(hex_array)

def hexStringToByteStrem(hex_string):
    hex_string = hex_string.replace(" ", "")
    return bytes.fromhex(hex_string)


# hex_array = [0x02, 0x01, 0x04, 0x00, 0x57, 0x50, 0x06, 0x00, 0xb2, 0x03]
# hex_string = "02 01 04 00 57 50 06 00 B2 03"

# hexArrayToByteStream(hex_array), hexStringToByteStrem(hex_string)

In [ ]:
def decode_weight(weight_bytes):
    weight = weight_bytes[0] + (weight_bytes[1] << 8) + (weight_bytes[2] << 16) + (weight_bytes[3] << 24)
    weight_in_grams = weight * 0.1
    return weight_in_grams
# client_socket, addr = server_socket.accept()
#########################################
def handle_client(client_socket):
    while True:
        try:
            # 接收客户端发送的消息receive message from client
            msg = client_socket.recv(1024)
            time1 = datetime.datetime.now()
            if msg:
                # 广播消息给所有其他客户端broadcast the message to all clients
                #broadcast_message(msg, client_socket)

                #print(time1)
                hex_a = list(msg)
                # print("[ %s ] receive from %s : %s" % (time1, addr,hex_a))

                # print("[ %s ] receive from %s : %s" % (time1, addr, hex_a[8:-2]))
                # print("[ %s ] receive from %s :%s | %s, %s" % (time1, addr,hex_a[8:-2],  decode_weight(hex_a[8:-2][1:5]), decode_weight(hex_a[8:-2][-4:])))
                # print("%s, %s" % decode_weight(hex_a[8:-2][1:5]), decode_weight(hex_a[8:-2][-4:]))
                print(f'hex_a: {hex_a} --weight 0: {decode_weight(hex_a[8:-2][1:5])}, weight 1: {decode_weight(hex_a[8:-2][-4:])}')
                # print(f'hex_a: {hex_a}')


                # print(client_sockets)

                #time.sleep(0.01)
                print("continuous...")

                response = hexArrayToByteStream(hex_array)
                # response = hexStringToByteStrem(hex_string)
                time2 = datetime.datetime.now()
                client_socket.sendall(response)

                # data_package = response[8:-2]

                # print("[ %s ] send to client: %s" %(time2, response))
        except:
            # socket。 when there is error, remove client and close socket.
            index = client_sockets.index(client_socket)
            client_sockets.remove(client_socket)
            client_socket.close()

            break

In [ ]:


#get parameter command
#hex_array = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x03, 0x00, 0xa9, 0x03]
#get aisle weight command

hex_array = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x06, 0x00, 0xac, 0x03]#get_w
# hex_array = [0x02, 0x01, 0x04, 0x00, 0x57, 0x50, 0x06, 0x00, 0xB2, 0x03]#tare
# 02 01 04 00 51 50 01 00 A7 03
#get loadcell weight command
#hex_array = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x07, 0x00, 0xad, 0x03]
#  TCP/IP socket Creat a TCP/IP socket
print("server ip:", HOST)
print("server port:", PORT)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#  binding sever address and port
server_socket.bind((HOST, PORT))

#  Start listening coming connection
server_socket.listen(2)
print('Waiting for client connection...')




# Store the list of sockets connected to clients
client_sockets = []
while True:
    #  wait for clients connection
    client_socket, addr = server_socket.accept()

    print('Connecting to client {0}'.format(addr))

    #add new client to list
    client_sockets.append(client_socket)

    # response = bytes(hex_array)
    response = hexArrayToByteStream(hex_array)
    # response = hexStringToByteStrem(hex_string)

    client_socket.sendall(response)
    print("Initial")
    print("send to client:", response)
    # creat a thread to handle message from clients
    thread = threading.Thread(target=handle_client, args=(client_socket,))
    thread.start()


    # hex_a: [2, 129, 12, 0, 81, 80, 6, 27, 231, 0, 0, 0, 0, 0, 0, 0, 0, 54, 3]
    # hex_a: [2, 129, 12, 0, 87, 80, 6, 27, 231, 27, 232, 0, 0, 0, 0, 0, 0, 0, 63, 3] ""





    





In [ ]:
[2,# start 
129, # id
84, 0, # frame length 
81, # command class Q
80, # cpmmand code
6, # parameter PN
20, #  aisle number mean total quantity of aisle in network
238, 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
168, # CS, 
3 # end
]

In [ ]:
# handle -ve values 

import socket
import threading
import datetime
import struct

# Specify server host and port
HOST = '192.168.0.129'
PORT = 8235

# Commands
GET_WEIGHT = [0x02, 0x01, 0x04, 0x00, 0x51, 0x50, 0x06, 0x00, 0xac, 0x03]
TARE = [0x02, 0x01, 0x04, 0x00, 0x57, 0x50, 0x06, 0x00, 0xB2, 0x03]

getweight_counter = 0

def hexArrayToByteStream(hex_array):
    return bytes(hex_array)

def decode_weight(weight_bytes):
    # Interpret bytes as a signed 32-bit integer
    weight = struct.unpack('<i', bytes(weight_bytes))[0]
    weight_in_grams = weight * 0.1
    return weight_in_grams

def send_command_and_handle_response(client_socket, command):
    global getweight_counter
    print(f"getweight_counter: {getweight_counter}")
    
    response = hexArrayToByteStream(command)
    client_socket.sendall(response)
    
    msg = client_socket.recv(1024)
    time1 = datetime.datetime.now()
    
    if msg:
        hex_a = list(msg)
        print(f'hex_a: {hex_a} --weight 0: {decode_weight(hex_a[8:-2][1:5])}, weight 1: {decode_weight(hex_a[8:-2][-4:])}')
        print("continuous...")
    
    if command == GET_WEIGHT:
        getweight_counter += 1

def handle_client(client_socket):
    global getweight_counter
    
    while True:
        try:
            if getweight_counter % 2500 ==0:  # Adjusted to 30 for demonstration
                print("Performing tare operation...")
                send_command_and_handle_response(client_socket, TARE)
                getweight_counter = 0
            
            send_command_and_handle_response(client_socket, GET_WEIGHT)
            
        except Exception as e:
            print(f"Error: {e}")
            if client_socket in client_sockets:
                client_sockets.remove(client_socket)
            client_socket.close()
            break

# The rest of your code remains the same
print("server ip:", HOST)
print("server port:", PORT)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(2)
print('Waiting for client connection...')

client_sockets = []

while True:
    client_socket, addr = server_socket.accept()
    print('Connecting to client {0}'.format(addr))
    client_sockets.append(client_socket)
    
    send_command_and_handle_response(client_socket, GET_WEIGHT)
    print("Initial")
    
    thread = threading.Thread(target=handle_client, args=(client_socket,))
    thread.start()
